In [2]:
# import os
# import sys

# module_path = os.path.abspath(os.path.join('../../src'))
# print("adding following folder to path: ",module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)

In [3]:
import numpy as np 
from numpy import linspace
import pandas as pd


import datetime
from scipy import stats
import time
import copy


import csv 
from pathlib import Path
from os import listdir
import pickle

from tabulate import tabulate


from datetime import datetime,timedelta

import itertools


import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from matplotlib.ticker import NullFormatter
from matplotlib.colors import Normalize
import matplotlib.pylab as pl
from matplotlib.collections import LineCollection

from pytz import timezone




In [4]:
from nna.pre_process_func import read_queue
from nna.fileUtils import read_file_properties
from nna.labeling_utils import load_labels
from nna.visUtils import get_cycle,createTimeIndex,file2TableDict,

In [4]:
# INPUT_DIR_PARENT = "/home/data/nna/stinchcomb/NUI_DATA/"


In [5]:
linestyle_str = [
     ('solid', 'solid'),      # Same as (0, ()) or '-'
     ('dotted', 'dotted'),    # Same as (0, (1, 1)) or '.'
     ('dashed', 'dashed'),    # Same as '--'
     ('dashdot', 'dashdot'),  # Same as '-.
     ('densely dotted',        (0, (1, 1))),
     ('densely dashed',        (0, (5, 1))),
    ('densely dashdotted',    (0, (3, 1, 1, 1))),
     ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1)))]


In [6]:
dataFlder = "/home/enis/projects/nna/data/"
resultsFlder = "/home/enis/projects/nna/results/"


In [7]:
id2name={}
id2name["_CABLE"]="Cable"
id2name["_RUNNINGWATER"]="Running Water"
id2name["_INSECT"]="Insect"
id2name["_RAIN"]="Rain"
id2name["_WATERBIRD"]="Water Bird"
id2name["_WIND"]="Wind"
id2name["_SONGBIRD"]="Songbird"
id2name["_AIRCRAFT"]="Aircraft"



In [8]:
# file_properties_df=pd.read_pickle("../../data/stinchcomb_dataV1.pkl")
file_properties_df=pd.read_pickle("../../data/prudhoeAndAnwr4photoExp_dataV1.pkl")

#important to keep them in order
file_properties_df.sort_values(by=['timestamp'],inplace=True)

# delete older than 2016
fromtime=datetime(2016, 1, 1, 0)
file_properties_df=file_properties_df[file_properties_df.timestamp>=fromtime]
all_areas=sorted(pd.unique(file_properties_df.site_id.values))

In [9]:
# import pandas as pd
# file_properties_df=pd.read_pickle("../../data/stinchcomb_dataV1.pkl")
# file_properties_df2=pd.read_pickle("../../data/realdata_v2No_stinchcomb.pkl")

In [10]:
# asd=pd.concat([file_properties_df,file_properties_df2])
# asd.sort_values(by=['timestamp'],inplace=True)

# asd.to_pickle("../../data/allFields_dataV3.pkl")


In [12]:
# !ls ../../data/

In [9]:
# file_properties_df[file_properties_df.site_id=="47"]

In [28]:
# PARAMS
# FREQS to reduce results 
# freq="30min"
# freq="2H"
freq="270min"
# freq="135min"
# freq="continous"



# possible places to pick
# sorted(pd.unique(file_properties_df.site_id.values))
# areas to be visualized



In [29]:

# globalindex,all_start,all_end=createTimeIndex(selected_areas,file_properties_df,freq)

# selected_tag_name="_SONGBIRD"



# weather_cols=[]

visFilePath="/home/enis/projects/nna/results/vis/PrudhoeAnwrV1/"


In [30]:
# all_areas

In [13]:
# print(" Duration of selected data period:",(all_end-all_start).days,"days")
# print(" Starts: {} \n Ends:   {}".format(all_start,all_end))

In [14]:
# globalindex

In [15]:
# file_properties_df[file_properties_df.site_id=="31"]

In [52]:

file_properties_df.iloc[-1]["timestamp"]

Timestamp('2019-09-09 14:18:02')

In [16]:
# file_properties_df[file_properties_df.site_id==selected_area]

In [57]:
from nna.fileUtils import standardPathStyle,list_files
from nna.visUtils import loadResults

In [59]:
def file2TableDict(selected_areas,model_tag_names,globalindex,globalcolumns,
                    file_properties_df,freq,dataFreq="10S",dataThreshold=0.5,
                    channel=1,gathered_results_perTag=None,
                    result_path=None,fileNameAddon="",prob2binaryFlag=True):
    # using gathered_results_perTag dictionary or  result_path to create
    # a pandas dataframe for visualizations

    # dataFreq is sampling frequency of the data,
    #most of the time we have predictions for each 10 second

    df_dict={key: None for (key) in selected_areas}
    no_result_paths=[]

    #we need to load it from files
    if gathered_results_perTag==None and (result_path==None):
        print("ERROR: gathered_results_perTag or (result_path and subDirectoryAddon )should be defined")
        return (None,None)


    for i,area in enumerate(selected_areas):
        df_sums = pd.DataFrame(index=globalindex, columns=globalcolumns).fillna(0)
        df_count = pd.DataFrame(index=globalindex, columns=globalcolumns).fillna(0)

        for modelTagName in model_tag_names:
    #         for afile in selected_areas_dict[area]:
            area_filtered=file_properties_df[file_properties_df.site_id==area]
            for afile,row in area_filtered.iterrows():
        #         data=gathered_results[afile][0]
                afile=Path(afile)
                # we either load data from multiple files or from single one
                if gathered_results_perTag==None:
                    # TODO, make _FCmodel variable
                    checkFolder=standardPathStyle(result_path,row,subDirectoryAddon=modelTagName
                                        ,fileNameAddon=fileNameAddon)
                    allSegments = list_files(str(checkFolder)+"/")
                    allSegments.sort()
                    if not allSegments:
                        data=np.empty(0)
                    else:
                        data=loadResults(allSegments,prob2binaryFlag=prob2binaryFlag,
                                        threshold=dataThreshold,channel=channel)
                        # gathered_results[file]=result[:]
                else:
                    data=gathered_results_perTag[modelTagName].get(afile,np.empty(0))[:]
                    if data.size!=0 and prob2binaryFlag==True:
                        data=prob2binary(data,threshold=0.5,channel=channel)

                if data.size==0:
                    no_result_paths.append(afile)
                    continue

                start=file_properties_df.loc[afile]["timestamp"]
                end =start+timedelta(seconds=(10*(len(data)-1)))
                index = pd.date_range(start,end, freq=dataFreq)
                df_afile=pd.DataFrame(data,index=index,columns=[modelTagName])
                # df_afile_grouped = df_afile.groupby([pd.Grouper(freq=freq)])
                # counts=df_afile_grouped.count()
                # sums=df_afile_grouped.sum()
                globalindexStart=globalindex.searchsorted(df_afile.index[0])
                globalindexStart= 0 if globalindexStart==0 else globalindexStart-1
                globalindexEnd=globalindex.searchsorted(df_afile.index[-1])
                globalindexEnd= globalindexEnd+1 if globalindexEnd==globalindexStart else globalindexEnd
#                 print(globalindexStart)
                theBins=pd.cut(df_afile.index,globalindex[globalindexStart:globalindexEnd+1])
                # theBins=pd.cut(df_afile.index,globalindex)
                df_afileGrouped=df_afile.groupby(theBins)
                sums=df_afileGrouped.agg("sum")
                counts=df_afileGrouped.agg("count")
                sums.set_index(sums.index.categories.left,inplace=True)
                counts.set_index(counts.index.categories.left,inplace=True)

                df_count=df_count.add(counts, fill_value=0) #df_count.update(counts)
                df_sums=df_sums.add(sums, fill_value=0) #df_sums.update(sums)

        df_dict[area]=(df_count.copy(),df_sums.copy())

    return df_dict,no_result_paths

In [66]:
result_path="/scratch/enis/data/nna/real/"
model_tag_names=["CABLE","RUNNINGWATER","INSECT", "RAIN", "WATERBIRD", "WIND", "SONGBIRD", "AIRCRAFT"]
def main():
    cmap = pl.cm.tab10
    aCmap = cmap
    my_cmaps = addNormalDistAlpha(aCmap)
    for selected_area in all_areas[:]:
        print(selected_area,all_areas.index(selected_area))
        nna.visutils.vis_preds_with_clipping(selected_area, file_properties_df, freq,model_tag_names, my_cmaps)
main()
# try:
#     main()
#     !conda run -n speechEnv python /home/enis/projects/nna/src/slack_message.py -m "continous figures ready "
# except Exception as ex:
#     print(ex)
#     !conda run -n speechEnv python /home/enis/projects/nna/src/slack_message.py -m "continous figures failed"


11 0
60 number of files do not have results
407 number of files do not have results


/scratch/enis/conda/envs/speechEnv/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


12 1
51 number of files do not have results
379 number of files do not have results
13 2
47 number of files do not have results
495 number of files do not have results
14 3
56 number of files do not have results
494 number of files do not have results
15 4
35 number of files do not have results
374 number of files do not have results
16 5
10 number of files do not have results
146 number of files do not have results
17 6
25 number of files do not have results
317 number of files do not have results
18 7
10 number of files do not have results
143 number of files do not have results
19 8
19 number of files do not have results
296 number of files do not have results
20 9
27 number of files do not have results
430 number of files do not have results
21 10
18 number of files do not have results
440 number of files do not have results
22 11
16 number of files do not have results
386 number of files do not have results
23 12
19 number of files do not have results
343 number of files do not ha

In [63]:

def VisPredsWClipping(selected_area,file_properties_df,freq,model_tag_names,my_cmaps):
    selected_areas=[selected_area,]
    # file length based time index
    if freq=="continous":
        fileTimeIndexSeries = getTimeIndexPerFile(selected_area,file_properties_df,freq)
        globalindex = fileTimeIndexSeries
    #fixed freq based  time index
    else:
        globalindex,all_start,all_end = createTimeIndex(selected_areas,file_properties_df,freq)

    selected_tag_name=["_"+i for i in model_tag_names]
    globalcolumns=selected_tag_name #selected_areas+weather_cols
    
#     print(globalindex)
    df_dict,no_result_paths = file2TableDict(selected_areas,selected_tag_name,globalindex,
                                             globalcolumns,file_properties_df,freq,dataFreq="10S",
                                             result_path=result_path,prob2binaryFlag=False)
    if len(no_result_paths)!=0:
        print("{} number of files do not have results".format(len(no_result_paths)))
#         print(no_result_paths[:1])
#         print(no_result_paths[-1:])

    
    regionName=file_properties_df[file_properties_df.site_id==selected_area][:1].region[0]

    # we are not using this for visualizations
    # df_dict_reverse=reverseTableDict(selected_areas,df_dict,model_tag_names)
    df_count,df_sums = df_dict[selected_area]

    df_freq=df_sums/df_count
    # del df_freq['UMIAT']
    df_freq=df_freq*100
    
    ########     LOAD Clipping     #########
    clippingResultsPath=dataFlder+"clipping_results_old/"
    selected_tag_name="Clipping"
#     model_tag_names=[selected_tag_name]
    globalcolumns=[selected_tag_name] #selected_areas+weather_cols

    gathered_results_perTag = loadClipping2Dict(clippingResultsPath,selected_areas,selected_tag_name)

    df_dict_clipping,no_result_paths = file2TableDict(selected_areas,globalcolumns,globalindex,globalcolumns,
                                    file_properties_df,freq,dataFreq="10S",dataThreshold=0.01,channel=2,
                                    gathered_results_perTag=gathered_results_perTag,result_path=None)

    df_count_clipping,df_sums_clipping = df_dict_clipping[selected_area]
    df_freq_clipping = df_sums_clipping / df_count_clipping
    df_freq_clipping = df_freq_clipping * 100

    ### add Clipping data to predictions 

    df_freq = pd.concat([df_freq, df_freq_clipping], axis=1, sort=False)
    if len(no_result_paths)!=0:
        print("{} number of files do not have results".format(len(no_result_paths)))

    ### Divide data into months

    cord_list=[(i,(0,0)) for i in df_freq.columns]
    
    monthsTime=pd.unique(df_freq.index.strftime("%Y-%m-01"))
    monthsTime=[pd.Timestamp(i) for i in monthsTime]
    
    monthsTimeStr=["{}-{}".format(month.year,month.month) for month in monthsTime]
    months=[df_freq.loc[month:month] for month in monthsTimeStr]
    ##### align all months
    for i,month in enumerate(months):
        months[i]=month.rename(index=lambda x: x.replace(month=7,year=2019))

    uniqueYears=np.unique([month.year for month in monthsTime])
    for year in uniqueYears:
        monthsInAYear=[months[i] for i,month in enumerate(monthsTime) if month.year==year]
        monthsTimeInAYear=[monthsTime[i] for i,month in enumerate(monthsTime) if month.year==year]
        createFigure(selected_area,monthsInAYear,monthsTimeInAYear,my_cmaps,cord_list,visFilePath,regionName,year,)



In [18]:
def getTimeIndexPerFile(selected_area,file_properties_df,freq,timeDifference=5):
    #TODO add sensitivity
    rowiterator=file_properties_df[file_properties_df.site_id==selected_area].iterrows()
    # use first item as initialization, ahead of for loop
    n=next(rowiterator)
    start=n[1].timestamp
    beginning=start
    end=n[1].timestampEnd
    fileTimeIndex=[]
    IsHead=False
#     print(start)
    for row in rowiterator:
        # if end of previous file not equal to start of the second one
        if (row[1].timestamp-end)>timedelta(minutes=timeDifference):
            # add previous one to list and make new one the beginning of continous recording
            fileTimeIndex.append(beginning)
            beginning=row[1].timestamp
            IsHead=True
    #             pass
#             print("noteq",row[1].timestamp-end)
    #         print(row[1].timestamp,end)
        # if they are equal, they should be in the same bin, so keep going
        else:
            IsHead=False
    #             pass
    #         print("equal",row[1].timestampEnd-start)
    #         fileTimeIndex.append(start)
    #         print(row[1].timestampEnd,start)
        start=row[1].timestamp
        end=row[1].timestampEnd
    # If last one is a head add to the list
#     print(IsHead)
    if IsHead:
        fileTimeIndex.append(beginning)
    # add end since last bin border should be bigger than all data
    fileTimeIndex.append(end)
    fileTimeIndexSeries=pd.Series(fileTimeIndex)
    return fileTimeIndexSeries


# 10191019101019101910101910191


In [19]:
def addNormalDistAlpha(aCmap):
    # Choose colormap
    # cmap = pl.cm.tab10
    cmap=aCmap
    # Get the colormap colors
    my_cmap = aCmap(np.arange(aCmap.N))
    my_cmaps=[]
    for clr in my_cmap:
        r,g,b,_=clr
        cdict = {'red':   [[0.0,  r, r],
                       [1.0,  r, r]],
             'green': [[0.0,  g,  g],
                       [1.0,  g, g]],
             'blue':  [[0.0,  b, b],
                       [1.0,  b, b]],
             'alpha':  [[0,  0.9, 0.9],
                       [1,  0.1, 0.1]]}

        newcmp = LinearSegmentedColormap('testCmap', segmentdata=cdict, N=100)
        my_cmaps.append(newcmp)
    return my_cmaps

def loadClipping2Dict(clippingResultsPath,selected_areas,selected_tag_name):
    gathered_results_perTag={selected_tag_name:{}}
    gathered_results={}
    selected_areas_files={}
    for i,area in enumerate(selected_areas):
        to_be_deleted=[]
        fileName=(clippingResultsPath+area+"_1.pkl")
        resultsDict=np.load(fileName,allow_pickle=True)
        resultsDict=resultsDict[()]
        gathered_results_perTag[selected_tag_name].update(resultsDict)
    return gathered_results_perTag


# def loadClipping(clippingInfoFile):
    
#     clippingInfo=np.load(clippingInfoFile,allow_pickle=True)
#     clippingInfo = clippingInfo[()]
#     clippingInfo2={}
#     clippingInfoArray=[]
#     cc=0
#     for clipFile,clipping in clippingInfo.items():
#         locID=Path(clipFile).stem.split("_")[:2]
#         locID = tuple(locID)
#         clippingInfo2[locID] = clipping
#         if clipping.shape==(1,2):
#             clipping = clipping[0]
#         elif clipping.shape==(2,2):
#             clipping = clipping[0]
#         elif clipping.shape==(2,):
#             pass
#         else:
#             print("ERROR",clipping)
#         clippingInfoArray.append(clipping)
#         clippingInfo2[locID] = clipping

#     clippingInfoArray = np.concatenate(clippingInfoArray).reshape(-1,2)
#     return clippingInfo2,clippingInfoArray


In [31]:
def createFigure(selected_area,months,monthsTime,my_cmaps,cord_list,visFilePath,regionName,year):
#     plt.rcParams["axes.prop_cycle"] = get_cycle("tab10",N=8)
    vmin,vmax=0,100
    normalize = Normalize(vmin=vmin, vmax=vmax)

    fig, ax = plt.subplots(figsize=(80,len(months)*9),nrows=len(months),sharex=True, sharey=True,gridspec_kw={'hspace': 0})
    ax = np.array(ax).reshape(-1) # subplot returns single element for single row

    markers = itertools.cycle((',', '+', '.', 'o', '*')) 

    weather_colors=["firebrick","darkorange","green","seagreen","lightpink"]

    for monthi,month in enumerate(months):
        # for col in df_freq.columns:
        for i,(col,(lat,long)) in enumerate(cord_list):
#             if col in weather_cols:
#                 index=weather_cols.index(col)
#                 ax[monthi].plot_date(month.index.to_pydatetime(), month[col],linestyle="-",marker=" ",color=weather_colors[index])
#             else:
#             ax[monthi].plot_date(month.index.to_pydatetime(), month[col],linestyle="-",marker=" ")
            if col=="Clipping":
                continue
            #convert dates to numbers first
            inxval = mdates.date2num(month[col].index.to_pydatetime())
            points = np.array([inxval, month[col].values]).T.reshape(-1,1,2)
            segments = np.concatenate([points[:-1],points[1:]], axis=1)
            lc = LineCollection(segments, cmap=my_cmaps[i],norm=normalize, linewidth=3,)
            # set color to date values
            lc.set_array(month["Clipping"])
            # note that you could also set the colors according to y values
            # lc.set_array(s.values)
            # add collection to axes
            ax[monthi].add_collection(lc)
#             break

    # add legend and set names of the lines
    ax[0].legend( labels=[id2name.get(x[0],x[0][1:]) for x in cord_list],loc='upper left', 
                borderpad=0.2, labelspacing=0.2, fontsize=28, 
                frameon=True) # frameon=False to remove frame.

    # set colours of the lines on the legend
    leg = ax[0].get_legend()
    for i,(col,(lat,long)) in enumerate(cord_list):
        if col=="Clipping":
            continue
        leg.legendHandles[i].set_color(my_cmaps[i](vmin)[:-1])


    ax[-1].set_xlabel('Day Number', fontsize=32)

#     uniqueYears=pd.unique([month.year for month in monthsTime])
#     uniqueYears.size

    for i,an_ax in enumerate(ax):    
        an_ax.set_ylabel('{}'.format(monthsTime[i].strftime("%Y-%B")),fontsize=48) #, fontweight='black')

        locator=mdates.DayLocator()
        an_ax.xaxis.set_minor_locator(locator)
        an_ax.xaxis.set_minor_formatter(mdates.DateFormatter('%d\n'))

        an_ax.xaxis.grid(True, which="minor")
        an_ax.xaxis.grid(True, which="major")


        an_ax.xaxis.set_major_locator(mdates.AutoDateLocator())
        an_ax.xaxis.set_major_formatter(mdates.DateFormatter('%d\n'))
        

        an_ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:.0f}'))
        an_ax.yaxis.grid()
        an_ax.tick_params(labelsize=22,which="minor")
        an_ax.tick_params(labelsize=25,which="major")
        
        # TODO figure out why we need to autoscale_view
        an_ax.autoscale_view()

    plt.tight_layout()
    plt.margins(x=0)
    plt.subplots_adjust(top=0.90)


    fig.suptitle('Site {}, Normalized Bi-270min Frequency [%]'.format(selected_area),fontsize=48)
#     plt.show()

    figDir= Path(visFilePath) / ("Freq-"+freq) / regionName 
    figDir.mkdir(parents=True,exist_ok=True)
    figPath= figDir / ("_".join([selected_area,str(year)]) +'.'+"png")
    
    fig.savefig(figPath)
#     fig.show()
#     fig.savefig("test" +'.png')
    plt.close(fig)
